In [ ]:
"""

Script to look at the application of classic parameterisations to Smith future run

Author: C. Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
import basal_melt_param.melt_functions as meltf
from basal_melt_param.constants import *
import seaborn as sns
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import glob

In [ ]:
%matplotlib qt5

In [ ]:
nemo_run = 'bf663'

In [ ]:
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
inputpath_mask = '/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'
inputpath_profiles = '/bettik/burgardc/DATA/NN_PARAM/interim/T_S_PROF/SMITH_'+nemo_run+'/'
outputpath_simple_all = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/SIMPLE/nemo_5km_06161821_oneFRIS/'
inputpath_plumes = '/bettik/burgardc/DATA/NN_PARAM/interim/PLUMES/SMITH_'+nemo_run+'/'
outputpath_plumes = '/bettik/burgardc/DATA/NN_PARAM/interim/PLUMES/SMITH_'+nemo_run+'/'
inputpath_boxes = '/bettik/burgardc/DATA/NN_PARAM/interim/BOXES/SMITH_'+nemo_run+'/'
outputpath_boxes = '/bettik/burgardc/DATA/NN_PARAM/interim/BOXES/SMITH_'+nemo_run+'/'
outputpath_melt = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/SMITH_'+nemo_run+'_CLASSIC/'
outputpath = '/bettik/burgardc/DATA/NN_PARAM/interim/'

In [ ]:
inputpath_mask = '/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_1980.nc').drop('time')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)
file_isf['isf_name'] = file_isf['isf_name'].where(file_isf['isf_name'] != 'Ekstrom', np.array('Ekström', dtype=object))
isf_names = file_isf['isf_name']

In [ ]:
isf_dim = large_isf.Nisf.drop_sel(Nisf=4)

In [ ]:
isf_dim

In [ ]:
#run_list = ['OPM031'] 
file_isf_list = []
target_melt_list = []
ds_melt_Gt_list = []
ds_melt_box1_list = []
ref_box1_list = []

for yy in tqdm(range(1980,1980 + 60)):
    
    # Param files
    melt_param_files = list(sorted(glob.glob(outputpath_melt+'eval_metrics_1D_*_oneFRIS_'+str(yy)+'.nc')))
    
    param_list = []
    for mfilename in melt_param_files:
        #paramname = mfilename[97:157].split('.')[0].split('_')[:-1]
        paramname = mfilename[89:157].split('.')[0].split('_')[:-1]
        #paramname = mfilename[105:157].split('.')[0].split('_')[:-1]
        mparam = '_'.join(paramname)
        param_list.append(mparam)
    
    ds_melt_param  = xr.open_mfdataset(melt_param_files, concat_dim='new_param', combine='nested', coords='minimal',compat='override')#, chunks={'x': chunksize, 'y': chunksize})
    if 'option' in ds_melt_param.coords:        
        ds_melt_param = ds_melt_param.drop('option')
    if 'calib' in ds_melt_param.coords: 
        ds_melt_param = ds_melt_param.drop('calib')
    if 'param' in ds_melt_param.coords: 
        ds_melt_param = ds_melt_param.drop('param')

    ds_melt_param = ds_melt_param.rename({'new_param': 'param'})
    ds_melt_param = ds_melt_param.assign_coords(param=param_list, time=yy)
    ds_melt_param_Gt = ds_melt_param['melt_1D_Gt_per_y'].sel(Nisf=ds_melt_param.Nisf)
    ds_melt_Gt_list.append(ds_melt_param_Gt)
    ds_melt_param_box1 = ds_melt_param['melt_1D_mean_myr_box1'].sel(Nisf=ds_melt_param.Nisf)
    ds_melt_box1_list.append(ds_melt_param_box1)
    
    # File for target
    NEMO_melt_rates_1D = xr.open_dataset(inputpath_data+'melt_rates_1D_NEMO_oneFRIS_'+str(yy)+'.nc')
    target_melt_Gt_yr = NEMO_melt_rates_1D['melt_Gt_per_y_tot'].sel(Nisf=ds_melt_param.Nisf)
    target_melt_list.append(target_melt_Gt_yr)
    NEMO_box1_myr = xr.open_dataset(inputpath_data+'melt_rates_box1_NEMO_oneFRIS_'+str(yy)+'.nc')
    ref_box1 = NEMO_box1_myr['mean_melt_box1_myr']
    ref_box1_list.append(ref_box1)
    
target_melt_all = xr.concat(target_melt_list, dim='time')
ref_box1_all = xr.concat(ref_box1_list, dim='time')
ds_melt_Gt_all = xr.concat(ds_melt_Gt_list, dim='time')
ds_melt_box1_all = xr.concat(ds_melt_box1_list, dim='time')


In [ ]:
np.sqrt(((ds_melt_Gt_all - target_melt_all)**2).mean(['time','Nisf'])).load() #.where(ds_melt_Gt_all.Nisf > 10, drop=True)

In [ ]:
np.sqrt(((ds_melt_Gt_all - target_melt_all)**2).where(ds_melt_Gt_all.Nisf > 11, drop=True).mean(['time','Nisf'])).load() #.where(ds_melt_Gt_all.Nisf > 10, drop=True)

In [ ]:
(ds_melt_Gt_all - target_melt_all).isel(param=1).plot()

In [ ]:
yend = 35

mean_10_param_list = []
mean_10_ref_list = []

for tstart in range(0,yend,10):
    mean_param_10yy = ds_melt_box1_all.sel(time=range(1970+tstart,1970+min(tstart+10,yend))).mean('time')
    mean_ref_10yy = ref_box1_all.sel(time=range(tstart,min(tstart+10,yend))).mean('time')
    
    mean_10_param_list.append(mean_param_10yy.assign_coords({'time': tstart}))
    mean_10_ref_list.append(mean_ref_10yy.assign_coords({'time': tstart}))

mean_10_param_box1 = xr.concat(mean_10_param_list, dim='time')
mean_10_ref_box1 = xr.concat(mean_10_ref_list, dim='time')


In [ ]:
np.sqrt(((mean_10_param_box1 - mean_10_ref_box1)**2).mean(['time','Nisf'])).load()